In [12]:
# import api keys
from api_keys import *
from IPython.display import JSON
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup as bs

**City Web Scraping**

In [13]:
url = ('https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom')
tables = pd.read_html(url, attrs={'class': 'wikitable sortable'})
type(tables)
#printing number of tables on wiki page
print(len(tables))

3


In [35]:
# selecting first table on page, as that is the one being referenced
cities= tables[0]
#dropping unnecessary columns
cities= cities.drop(cities.columns[[1,2,3,5]], axis=1)
#renaming columns
cities.rename(columns={'City[3][1]':'City', 'Nation/region':'Country'}, inplace=True)

In [15]:
#cleaning City column
import re
cities['City']= cities['City'].str.replace(r"\(.*|\d.*|\,.*|City of ", "", regex=True)

In [16]:
#cleaning Country column

cities['Country']=cities['Country'].str.replace(r".*\, ", '', regex=True)

In [17]:
#adding iso column and City_Country column

cities['ISO'] = pd.Series(["GB" for x in range(len(cities.index))])
cities['City_Country']= cities['City'] + ', ' + cities['ISO']

In [34]:
#cleaning Population column

cities['Population']=cities['Population'].str.replace(r"\,| .*|\[.*", '', regex=True)

Merging *cities* with *airports*

In [20]:
airports= (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(City_Country = lambda x: x['municipality'] + ', ' + x['iso_country'])
)

In [21]:
cities_airports= cities.merge(airports, how='inner').drop(['ISO', 'latitude_deg', 'longitude_deg', 'iso_country', 'iso_region', 'municipality', 'iata_code', 'Population', 'City', 'Country'], axis=1)


In [22]:
cities_airports.rename(columns={'name':'Airport', 'icao_code':'ICAO'}, inplace=True)

**Weather Forcast Data**

In [24]:
city = "London"
country = "GB"

response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
#JSON(response.json())

In [ ]:
forecast_api = response.json()['list']

weather_info = []

for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['Date_Time'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['Temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['Wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['Prob_Percipitation'] = float(forecast_3h['pop'])
    except: weather_hour['Prob_Percipitation'] = 0
    # rain
    try: weather_hour['Rain'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['Rain'] = 0
    # wind 
    try: weather_hour['Snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['Snow'] = 0
    weather_hour['City_Country'] = city + ', ' + country
    weather_info.append(weather_hour)
    
weather = pd.DataFrame(weather_info)

**Flight Arrival Data**

In [26]:
from datetime import datetime, timedelta

airport_icoa = "EGLL"
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=9)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"


import requests

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
#response.json()

In [27]:
import pandas as pd
arrivals = pd.json_normalize(response.json()['arrivals'])

arrivals = (arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal','arrival.terminal', 'status', 'number'])
    .assign(ICAO = airport_icoa)
    
)
arrivals.rename(columns={'departure.airport.name':'Departure_City', 'arrival.scheduledTimeLocal': 'Arrival_Time', 'arrival.terminal': 'Terminal', 'status': 'Status', 'number': 'Flight'}, inplace=True)

**Check tables**

In [28]:
cities.head()

,City,Country,Population,ISO,City_Country
0,Aberdeen,Scotland,22279338,GB,"Aberdeen, GB"
1,Armagh,Northern Ireland,14777,GB,"Armagh, GB"
2,Bangor,Wales,18808,GB,"Bangor, GB"
3,Bath,England,88859,GB,"Bath, GB"
4,Belfast,Northern Ireland,333871,GB,"Belfast, GB"


In [29]:
cities_airports.head()

,City_Country,Airport,ICAO
0,"Belfast, GB",Belfast International Airport,EGAA
1,"Birmingham, GB",Birmingham International Airport,EGBB
2,"Bristol, GB",Bristol Airport,EGGD
3,"Cardiff, GB",Cardiff International Airport,EGFF
4,"Edinburgh, GB",Edinburgh Airport,EGPH


In [30]:
weather.head()

,Date_Time,Temperature,Wind,Prob_Percipitation,Rain,Snow,City_Country
0,2022-08-24 18:00:00,28.01,2.66,0.00,0.00,0,"London, GB"
1,2022-08-24 21:00:00,25.13,1.08,0.00,0.00,0,"London, GB"
2,2022-08-25 00:00:00,21.95,1.33,0.04,0.00,0,"London, GB"
3,2022-08-25 03:00:00,20.21,1.12,0.67,1.22,0,"London, GB"
4,2022-08-25 06:00:00,19.22,2.05,0.92,2.72,0,"London, GB"


In [31]:
arrivals.head()

,Departure_City,Arrival_Time,Terminal,Status,Flight,ICAO
0,Glasgow,2022-08-24 15:05+01:00,5,Delayed,BA 1499,EGLL
1,Nice,2022-08-24 16:35+01:00,3,Expected,AF 7380,EGLL
2,Nápoli,2022-08-24 16:40+01:00,3,Expected,BA 2609,EGLL
3,Thessaloniki,2022-08-24 16:05+01:00,5,Delayed,BA 2643,EGLL
4,Preveza/Lefkada,2022-08-24 16:15+01:00,5,Delayed,BA 681,EGLL


**Connect Python to MySQL**

In [32]:
schema="gans"
host="data-eng-project.crq4hzbbr6f3.eu-west-2.rds.amazonaws.com"
user="xxx"
password="xxx"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities.to_sql('cities', if_exists='append', con=con, index=False)
cities_airports.to_sql('cities_airports', if_exists='append', con=con, index=False)
weather.assign(Date_Time = lambda x: pd.to_datetime(x['Date_Time'])).to_sql('weather', if_exists='append', con=con, index=False)
arrivals.to_sql('arrivals', if_exists='append', con=con, index=False)